In [1]:
import pandas as pd
import numpy as np
from ipywidgets import FloatProgress
from IPython.display import display

In [2]:
workingData = pd.read_csv('data/WorkingData.csv')
contests = workingData
print("There are " + str(len(contests)) + " Contests")

There are 372211 Contests


### Drop unecessary columns

In [3]:
categories = contests.columns.drop(['ContestName', 'GameSet', 'is_vault', 'DistinctUsers'])## AND ENTRIES!
contests = workingData[categories].set_index('ContestId')

In [4]:
contests.head()

,SportName,VariantName,ContestStartDatetimeEST,ContestEndDatetimeEST,ContestPayoutDatetimeEST,EntryFeeAmount,TotalPrizeAmount,MaxNumberPlayers,MaxEntriesPerUser,Entries,Contest_Group,NumGames,DraftablePlayersInSet,PaidUsersInDraftGroup,TopPrize,MaxPayoutPosition
ContestId,,,,,,,,,,,,,,,,
7963004,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,0.25,25000.0,117500,100,117500.0,Headliner,15,910.0,668396,2000.0,23665
7963437,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,2.00,25000.0,14375,100,14375.0,Booster,15,910.0,668396,20.0,1250
8296224,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,100.00,5000.0,55,1,55.0,SingleEntry,15,910.0,668396,1100.0,11
8296544,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,50.00,5000.0,111,11,111.0,FeaturedDoubleUp,15,910.0,668396,100.0,50
8494213,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,5.00,300000.0,69000,200,57604.0,Headliner,15,910.0,668396,25000.0,13805


### Adding Threshold

In [5]:
backup = contests

In [6]:
contests = backup

In [7]:
contests = contests.assign(SuccessThreshold= lambda x:(contests['TotalPrizeAmount'].astype(float)/(contests['EntryFeeAmount'].astype(float))).astype(int))

### Adding Success (and removing Entries)

In [8]:
contests = contests.assign(Success = lambda x: (x.Entries > x.SuccessThreshold).astype(float))

In [9]:
contests = contests.drop(columns=['Entries'], axis=1)

### Categorizing

In [10]:
categoricalFeatures = ['SportName', 'VariantName', 'Contest_Group']
def categorizeFeature(df, name):
    cats = {}
    for cat in workingData[name].unique():
        cats[name] = (str(name)+'_'+str(cat))
        df = df.assign(newCol = lambda x: (df[name] == cat).astype(float)).rename(columns = {'newCol':cats[name]})    
    df = df.drop(columns=[name], axis=1)
    return df

cBar = FloatProgress(min=0, max=len(categoricalFeatures))
display(cBar)

for f in categoricalFeatures:
    cBar.value += 1
#     print("Categorizing", f)
    contests = categorizeFeature(contests, f)

FloatProgress(value=0.0, max=3.0)

Categorizing SportName
Categorizing VariantName
Categorizing Contest_Group


In [11]:
contests.head(1)

,ContestStartDatetimeEST,ContestEndDatetimeEST,ContestPayoutDatetimeEST,EntryFeeAmount,TotalPrizeAmount,MaxNumberPlayers,MaxEntriesPerUser,NumGames,DraftablePlayersInSet,PaidUsersInDraftGroup,...,Contest_Group_MultiPlayer,Contest_Group_DoubleUp,Contest_Group_Satellite,Contest_Group_Qualifier,Contest_Group_50/50,Contest_Group_H2H,Contest_Group_LiveEvent,Contest_Group_3Max,Contest_Group_Multiplier,Contest_Group_DFSLeagues
ContestId,,,,,,,,,,,,,,,,,,,,,
7963004,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,0.25,25000.0,117500,100,15,910.0,668396,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Time

In [12]:
backup = contests

In [13]:
contests = backup
timeFeatures = ['ContestStartDatetimeEST', 'ContestEndDatetimeEST', 'ContestPayoutDatetimeEST']
timeNames = ['_Minute', '_Hour', '_Day', '_Month', '_DayOfWeek', '_DayOfYear', '_WeekOfYear', '_Year']
formatString = '%Y-%m-%d %H:%M:%S'

def mapTime(df, name, colHeader):
    df = df.assign(timeCol = lambda x: pd.to_datetime(df[name], format=formatString))
    df[colHeader+'_Minute'] = df.timeCol.apply(lambda x: x.strftime('%M')).astype(float)
    df[colHeader+'_Hour'] = df.timeCol.apply(lambda x: x.strftime('%H')).astype(float)
    df[colHeader+'_Day'] = df.timeCol.apply(lambda x: x.strftime('%d')).astype(float)
    df[colHeader+'_Month'] = df.timeCol.apply(lambda x: x.strftime('%m')).astype(float)
    df[colHeader+'_DayOfWeek'] = df.timeCol.apply(lambda x: x.strftime('%w')).astype(float)
    df[colHeader+'_DayOfYear'] = df.timeCol.apply(lambda x: x.strftime('%-j')).astype(float)
    df[colHeader+'_WeekOfYear'] = df.timeCol.apply(lambda x: x.strftime('%U')).astype(float)
    df[colHeader+'_Year'] = df.timeCol.apply(lambda x: x.strftime('%Y')).astype(float)

    df = df.drop(columns = [name, 'timeCol'], axis=1)
    return df


def scaleTime(df, colHeader):
    timeCols = []
#     for c in 
    df[colHeader+'_Minute_Scaled'] = df[colHeader+'_Minute'].apply(lambda x: x/60)
    df[colHeader+'_Hour_Scaled'] = df[colHeader+'_Hour'].apply(lambda x: x/24)
    df[colHeader+'_Day_Scaled'] = df[colHeader+'_Day'].apply(lambda x: x/31)
    df[colHeader+'_Month_Scaled'] = df[colHeader+'_Month'].apply(lambda x: x/12)
    df[colHeader+'_DayOfWeek_Scaled'] = df[colHeader+'_DayOfWeek'].apply(lambda x: x/7)
    df[colHeader+'_DayOfYear_Scaled'] = df[colHeader+'_DayOfYear'].apply(lambda x: x/365)
    df[colHeader+'_WeekOfYear_Scaled'] = df[colHeader+'_WeekOfYear'].apply(lambda x: x/52)
    df[colHeader+'_Year_Scaled'] = df[colHeader+'_Year'].apply(lambda x: (x-2000)/20)
    
    dropCols = []
    for n in timeNames:
        dropCols.append(colHeader+n)
    df = df.drop(columns = dropCols, axis=1)
    return df

import math
circularizeFeatures = ['_Minute_Scaled', '_Hour_Scaled', '_Day_Scaled', '_Month_Scaled', '_DayOfWeek_Scaled', '_DayOfYear_Scaled', '_WeekOfYear_Scaled']
def circularizeTime(df, colHeader):
    for f in circularizeFeatures:
        fStr = f[:-len('Scaled')]
        df = df.assign(X= lambda c: np.cos(df[colHeader+f]*2*math.pi)).assign(Y= lambda c: np.sin(df[colHeader+f]*2*math.pi))
        df = df.rename(columns={'X': colHeader+fStr+'X', 'Y': colHeader+fStr+'Y'}).drop(columns=[colHeader+f], axis=1)
    return df

tBar = FloatProgress(min=0, max=len(timeFeatures))
display(tBar)
for f in timeFeatures:
    newColHeader = f[len('Contest'):-len('DatetimeEST')]
#     print("Time data from", f)
    contests = mapTime(contests, f, newColHeader)
#     print("Scale times from", f)
    contests = scaleTime(contests, newColHeader)
#     print("Splitting x,y times from", f)
    contests = circularizeTime(contests, newColHeader)
    
    tBar.value+=1

FloatProgress(value=0.0, max=3.0)

### Time Series Data

In [14]:
backup = contests

In [15]:
contests.head()

,EntryFeeAmount,TotalPrizeAmount,MaxNumberPlayers,MaxEntriesPerUser,NumGames,DraftablePlayersInSet,PaidUsersInDraftGroup,TopPrize,MaxPayoutPosition,SuccessThreshold,...,Payout_Day_X,Payout_Day_Y,Payout_Month_X,Payout_Month_Y,Payout_DayOfWeek_X,Payout_DayOfWeek_Y,Payout_DayOfYear_X,Payout_DayOfYear_Y,Payout_WeekOfYear_X,Payout_WeekOfYear_Y
ContestId,,,,,,,,,,,,,,,,,,,,,
7963004,0.25,25000.0,117500,100,15,910.0,668396,2000.0,23665,100000,...,-0.994869,0.101168,-1.836970e-16,-1.0,-0.222521,0.974928,-0.267814,-0.963471,-0.239316,-0.970942
7963437,2.00,25000.0,14375,100,15,910.0,668396,20.0,1250,12500,...,-0.994869,0.101168,-1.836970e-16,-1.0,-0.222521,0.974928,-0.267814,-0.963471,-0.239316,-0.970942
8296224,100.00,5000.0,55,1,15,910.0,668396,1100.0,11,50,...,-0.994869,0.101168,-1.836970e-16,-1.0,-0.222521,0.974928,-0.267814,-0.963471,-0.239316,-0.970942
8296544,50.00,5000.0,111,11,15,910.0,668396,100.0,50,100,...,-0.994869,0.101168,-1.836970e-16,-1.0,-0.222521,0.974928,-0.267814,-0.963471,-0.239316,-0.970942
8494213,5.00,300000.0,69000,200,15,910.0,668396,25000.0,13805,60000,...,-0.994869,0.101168,-1.836970e-16,-1.0,-0.222521,0.974928,-0.267814,-0.963471,-0.239316,-0.970942


### Scale Features

In [16]:
scaleFeatures = ['EntryFeeAmount', 'TotalPrizeAmount', 'MaxNumberPlayers', 'MaxEntriesPerUser', 'NumGames', 'DraftablePlayersInSet', 'PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition', 'SuccessThreshold']

In [17]:
contests = backup[:]

In [18]:
def scaleFeature(df, name):
    minVal = df[name].min()
    maxVal = df[name].max()
    return df.assign(newCol=lambda x: (x[name]-df[name].min())/(df[name].max()-df[name].min())).rename(columns={'newCol': name+'_Scaled'}).drop(columns=[name], axis=1)

In [19]:
sBar = FloatProgress(min=0, max=len(timeFeatures))
display(sBar)

for f in scaleFeatures:
    print('Scaling', f)
    contests = scaleFeature(contests, f)
    sBar.value +=1

FloatProgress(value=0.0, max=3.0)

Scaling EntryFeeAmount
Scaling TotalPrizeAmount
Scaling MaxNumberPlayers
Scaling MaxEntriesPerUser
Scaling NumGames
Scaling DraftablePlayersInSet
Scaling PaidUsersInDraftGroup
Scaling TopPrize
Scaling MaxPayoutPosition
Scaling SuccessThreshold


### Summary and Saving

In [21]:
for c in contests.columns:
    print(c)

Success
SportName_NFL
SportName_PGA
SportName_SOCC
SportName_MLB
SportName_NAS
SportName_CFB
SportName_MMA
SportName_LOL
SportName_NHL
SportName_NBA
SportName_SOC
SportName_CBB
SportName_GOLF
SportName_CFL
SportName_EL
SportName_TEN
VariantName_Classic
VariantName_Weekend
VariantName_Arcade
VariantName_WNBA
VariantName_Tiers
VariantName_Playoff Arcade
VariantName_Late Swap
VariantName_Final Series
VariantName_Tiers Late Swap
VariantName_Showdown
VariantName_Match Play
VariantName_Showdown Captain Mode
VariantName_Lowball
VariantName_Single Match
Contest_Group_Headliner
Contest_Group_Booster
Contest_Group_SingleEntry
Contest_Group_FeaturedDoubleUp
Contest_Group_MultiPlayer
Contest_Group_DoubleUp
Contest_Group_Satellite
Contest_Group_Qualifier
Contest_Group_50/50
Contest_Group_H2H
Contest_Group_LiveEvent
Contest_Group_3Max
Contest_Group_Multiplier
Contest_Group_DFSLeagues
Start_Year_Scaled
Start_Minute_X
Start_Minute_Y
Start_Hour_X
Start_Hour_Y
Start_Day_X
Start_Day_Y
Start_Month_X
Start

In [22]:
# contests.to_csv("data/WorkedData.csv")


In [27]:
successes = len(contests.loc[contests['Success'] == 1])
fails = len(contests.loc[contests['Success'] == 0])

In [28]:
print("Successes:", successes)
print("Fails:", fails)

Successes: 355740
Fails: 16471


In [29]:
fails/(successes+fails)

0.04425178191939518